# PRÁCTICA 2: Clasificador de noticias

### Nombres:
Introduce en esta celda los nombres de los dos integrantes del grupo:\
*Alumno 1:* DANIEL CARMONA PEDRAJAS
*Alumno 2:* JOEL PARDO FERRERA

Objetivo: Implementar un clasificador usando el conjunto de datos recopilado de varias fuentes de internet como:

- Google News que toma noticias de varios repositorios dedicados a la información
- Periodicos:
    - El pais
    - ABC
    - El confidencial
    - 20minutos
    - El diario

Este repositorio incluye tanto las noticias en formato '.txt' donde se almacenan los cuerpos de noticia y sus correspondientes titulos, como un '.csv' donde se contiene un registro de todas las noticias donde se refleja el número de noticias, la clase a la que pertenece (deportes, salud, ciencia y politica), el número de noticia dentro de la clase correspondiente, el titulo de noticia, la ruta donde esta almacenada esa noticia, y por ultimo la URL correspondiente del repositorio de donde se ha sacado al noticia. 

La fechas fechas tanto de publicacion como de obtenciond e datos se ubican en Noviembre de 2022. 

La clase a predecir es el tipo de noticia (columna 'category' de la base de datos), a partir de los archivos '.txt'. 

## 1. IMPORTACION DE LIBRERIAS

In [ ]:
# git config --global user.email "jpardo0824@gmail.com"
# git config --global user.name "JPardo08"

In [7]:
!pip3 install pandas
!pip3 install spacy
!pip3 install scikit-learn

  Using cached scikit_learn-1.2.0-cp39-cp39-macosx_10_9_x86_64.whl (9.1 MB)
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)
  Using cached scipy-1.9.3-cp39-cp39-macosx_10_9_x86_64.whl (34.3 MB)
  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)


In [8]:
import pandas as pd
import numpy as np
import spacy
import unicodedata
#from spellchecker import SpellChecker 
#from textblob import TextBlob 
#import contractions
import re
import random
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.svm import SVC

## 2. CARGA DE DATOS
Cargamos los datos en dos formatos:
* DataFrame de pandas
* Generador

Para cargar los datos utilizamos la libreria Pandas. En la funcion impleentada, le pasamos unicmanete  la ruta del archivo csv que queramos cargar y se guarda los datos en una variable generador.

In [13]:
def parse(path):
    # Leemos el registro
    df = pd.read_csv(path,';')

    for i in df['index']:
    #Seleccionamos lo que nos interesa de cara a un futuro

    #Cogemos las columnas que nos interesan
        path1 = df.loc[i,'path'] # Columna de la ruta

        category = df.loc[i,'category'] # Columna sub
        
        with open(path1, 'r') as f:
            extracto = f.readline()


    
        yield path1, category, extracto
    
df_gen = parse('./TextClassification/Datos/urls1.csv')
df = pd.read_csv('./TextClassification/Datos/urls1.csv',',')
#Visualizamos los datos
df.head()

/var/folders/yq/prfd22yn0k3gnlt89mfz3zb80000gn/T/ipykernel_2165/889524856.py:21: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  df = pd.read_csv('./TextClassification/Datos/urls1.csv',';')


FileNotFoundError: [Errno 2] No such file or directory: './TextClassification/Datos/urls1.csv'

In [3]:
#Comprobamos los tipos
print(type(df))
print(type(df_gen))

<class 'pandas.core.frame.DataFrame'>
<class 'generator'>


In [4]:
#Miramos los nombres
df.columns

Index(['index', 'noticia', 'link_noticia', 'web', 'usuario', 'id_usuario',
       'fecha_envio', 'fecha_publicacion', 'meneos', 'clicks', 'comentarios',
       'votos_positivos', 'votos_anonimos', 'votos_negativos', 'karma', 'sub',
       'extracto'],
      dtype='object')

In [5]:
#Generamos un duplicado para seleccionar lo que queremos
df2 = df.loc[:,['noticia', 'sub','extracto']]

#Eliminamos los na
df2=df2.dropna()

In [6]:
#Comprobamos cuantas fila con na hemos eliminado
print(df.shape)
print(df2.shape)

(177509, 17)
(177363, 3)


In [7]:
#Comprobamos los valores de la columna sub
clases = pd.unique(df2['sub']).tolist()
clases

['cultura',
 'tecnología',
 'ocio',
 'actualidad',
 'mnm',
 'Japón',
 'escombrillos',
 'BUAMBUSUB',
 'ciencia',
 'Artículos',
 'Sucesos',
 'softlibre',
 'TeRespondo',
 'Biologia',
 'linux',
 'BARCOS',
 'Música',
 'AEDE',
 'Autorrelatos',
 'gilipolleces',
 'Videojuegos',
 'astronomia',
 'opinion',
 'idiomas',
 'Cocíname',
 'movilidad',
 'OYOYOY',
 'Cine',
 'Ochenteando',
 'Psicópatas',
 'Etimologia',
 'SysDevs',
 'Hemeroteca',
 'Pregúntame',
 'Ubuntu',
 'trenes',
 'Poesia',
 'Historia',
 'Neurociencia',
 'mexico',
 'MADRID',
 'PREVENCIÓN',
 'Galicia',
 'retrocomp',
 'INTERNAC',
 'Venezuela',
 'Abuso_Animal',
 'Jurídicas',
 'medicina',
 'Visualdata',
 'autonomos',
 'Series',
 'aviones',
 'Cantabria',
 'Bulos',
 'salud',
 'LGBT',
 'SEXOLOGÍA',
 'Alimentación',
 'Drones',
 'ACME',
 'Documentales',
 'Cómics',
 'RaspberryPi',
 'Libros',
 'netsec',
 'Podemos',
 'mascotas',
 'CiFi',
 'Crianza',
 'Encuestas',
 'UHF',
 'Economía',
 'Cuba',
 'GNU_LINUX',
 'MAmbiente',
 'Barcelona',
 'Fakeame',
 '

In [8]:
#Vamos a ver que clases tienen más peso
frecuencias = df2.groupby('sub').count()
frecuencias = frecuencias.sort_values('noticia',ascending=False)

#Visualizamos las 5 clases con mas peso
frecuencias.head()

,noticia,extracto
sub,,
mnm,135102,135102
actualidad,23933,23933
cultura,9544,9544
tecnología,4415,4415
ocio,3435,3435


In [9]:
#Visualizamos las 5 clases con menos peso
frecuencias.tail(5)

,noticia,extracto
sub,,
Ubuntu,1,1
EleccionesE,1,1
IMGUR,1,1
ELBAÚL,1,1
ACME,1,1


In [10]:
#Calculo de peso total de los datos.
numero_clases_total = len(pd.unique(df2['sub']))
print(numero_clases_total)

#Numero total de noticias.
numero_textos_total = df2.shape[0]
print(numero_textos_total)

114
177363


In [11]:
#Creamos un data frame para visualizar los datos.
frecuencias2 = pd.DataFrame()

In [12]:
#Vamos a ver todos los diferentes numeros de noticias. 
numero_textos_clase = pd.unique(frecuencias['noticia']).tolist() #(numero noticias)
numero_textos_clase = sorted(numero_textos_clase)
print(numero_textos_clase)

[1, 2, 3, 4, 5, 8, 9, 10, 11, 12, 13, 15, 16, 21, 22, 27, 30, 34, 36, 43, 55, 63, 77, 102, 111, 3435, 4415, 9544, 23933, 135102]


In [13]:
#Asignacion:
frecuencias2['Numero_Noticias'] = numero_textos_clase

#Comprobacion:
frecuencias2

,Numero_Noticias
0,1
1,2
2,3
3,4
4,5
5,8
6,9
7,10
8,11
9,12


In [14]:
#Mostramos para cada clase cuantas noticias tienen. 
clases_agrupa_nnc = []
numeros_clases_numero_texto = [] # (numero noticias /clase)
for i in list(numero_textos_clase):
        a = frecuencias[frecuencias['noticia']==i]
        noticias = a.index.values
        numeros_clases_numero_texto.append(len(noticias))
        clases_agrupa_nnc.append(noticias)
        print(a, '\n'*4)

              noticia  extracto
sub                            
Cómics              1         1
deportes            1         1
ebola               1         1
Documentales        1         1
Cuba                1         1
aviones             1         1
gatos               1         1
INTERNAC            1         1
mexico              1         1
CiFi                1         1
Cantabria           1         1
mascotas            1         1
medicina            1         1
Biologia            1         1
movilidad           1         1
netsec              1         1
opinion             1         1
retrocomp           1         1
smartphone          1         1
Autorrelatos        1         1
Animales            1         1
veganismo           1         1
autonomos           1         1
Economía            1         1
android             1         1
Filosofía           1         1
KKKantiPDMS         1         1
Hazlotumismo        1         1
Libros              1         1
MAmbient

In [15]:
#Asignacion:
frecuencias2['Clases'] = clases_agrupa_nnc
frecuencias2['Numero_Clases'] = numeros_clases_numero_texto
frecuencias2['Peso_Relativo'] = frecuencias2['Numero_Noticias'] * frecuencias2['Numero_Clases']
frecuencias2['Peso_Total_Clase%'] = frecuencias2['Peso_Relativo']/numero_textos_total * 100

#Comprobacion:
frecuencias2

,Numero_Noticias,Clases,Numero_Clases,Peso_Relativo,Peso_Total_Clase%
0,1,"[Cómics, deportes, ebola, Documentales, Cuba, ...",55,55,0.031010
1,2,"[Drones, Etimologia, museo, Encuestas, AEDE, B...",12,24,0.013532
2,3,"[Ochenteando, Clicómics, Drogolegas, MADRID]",4,12,0.006766
3,4,"[UHF, LGBT, Jurídicas, Лhcdss, Cine]",5,20,0.011276
4,5,"[Palabros, Series, Sucesos, SysDevs, Japón, Cr...",7,35,0.019734
5,8,"[gilipolleces, Cocíname, Preguntas]",3,24,0.013532
6,9,[dmnm],1,9,0.005074
7,10,"[idiomas, Música, softlibre]",3,30,0.016914
8,11,[trenes],1,11,0.006202
9,12,[Fakeame],1,12,0.006766


In [16]:
#Comprobacion:
print(len(clases_agrupa_nnc) == len(pd.unique(frecuencias['noticia'])))
print(frecuencias2['Peso_Relativo'].sum() == numero_textos_total)
print(frecuencias2['Peso_Total_Clase%'].sum() == 100)

True
True
True


In [17]:
#Ordenamos:
frecuencias2 = frecuencias2.sort_values('Peso_Total_Clase%',ascending=False)
frecuencias2

,Numero_Noticias,Clases,Numero_Clases,Peso_Relativo,Peso_Total_Clase%
29,135102,[mnm],1,135102,76.172595
28,23933,[actualidad],1,23933,13.493795
27,9544,[cultura],1,9544,5.381055
26,4415,[tecnología],1,4415,2.489245
25,3435,[ocio],1,3435,1.936706
24,111,[ciencia],1,111,0.062584
23,102,[emnm],1,102,0.057509
22,77,[astronomia],1,77,0.043414
21,63,[TeRespondo],1,63,0.035520
20,55,[Pregúntame],1,55,0.031010


Como podemos observar, unicamente las 3 primeras clases se llevan un 95,04% de los datos. Buscamos un conjunto de datos homogeneo para el cual hacer la prediccion por lo que vamos a descartar varias clases.

In [18]:
#Seleccion de entre un 0.01 hasta un 0.07 incluyendo ambos: 
condicion_si = (frecuencias2['Peso_Total_Clase%'] >= 0.0064) & (frecuencias2['Peso_Total_Clase%'] <= 2)


In [19]:
#Resumen datos clasificacion:
frecuencias3 =  frecuencias2[condicion_si]
frecuencias3

,Numero_Noticias,Clases,Numero_Clases,Peso_Relativo,Peso_Total_Clase%
25,3435,[ocio],1,3435,1.936706
24,111,[ciencia],1,111,0.062584
23,102,[emnm],1,102,0.057509
22,77,[astronomia],1,77,0.043414
21,63,[TeRespondo],1,63,0.035520
20,55,[Pregúntame],1,55,0.031010
0,1,"[Cómics, deportes, ebola, Documentales, Cuba, ...",55,55,0.031010
19,43,[Podemos],1,43,0.024244
13,21,"[linux, BARCOS]",2,42,0.023680
18,36,[Hemeroteca],1,36,0.020297


In [20]:
#Cogemos la lista con las clases disponibles.
disponibles = [] #Las clases que clasificamos 

for i in frecuencias3['Clases']:
    disponibles.extend(i)

print('Disponibles: \n')
print(disponibles)

print('\n'*4)

no_disponibles = []#Las clases que eliminamos 
for i in clases:
    if i not in disponibles:
        no_disponibles.append(i)
        
print('No disponibles: \n')
print(no_disponibles)

Disponibles: 

['ocio', 'ciencia', 'emnm', 'astronomia', 'TeRespondo', 'Pregúntame', 'Cómics', 'deportes', 'ebola', 'Documentales', 'Cuba', 'aviones', 'gatos', 'INTERNAC', 'mexico', 'CiFi', 'Cantabria', 'mascotas', 'medicina', 'Biologia', 'movilidad', 'netsec', 'opinion', 'retrocomp', 'smartphone', 'Autorrelatos', 'Animales', 'veganismo', 'autonomos', 'Economía', 'android', 'Filosofía', 'KKKantiPDMS', 'Hazlotumismo', 'Libros', 'MAmbiente', 'Galicia', 'Natación', 'Neurociencia', 'GNU_LINUX', 'OrienteMedio', 'PREVENCIÓN', 'Fotomundo', 'Poesia', 'Firefox', 'Psicópatas', 'alas', 'QUEJAS', 'RPDC', 'RaspberryPi', 'Rusia', 'SEXOLOGÍA', 'STARWARS', 'Feminismo', 'Eurovision', 'Entrevistas', 'Ubuntu', 'EleccionesE', 'IMGUR', 'ELBAÚL', 'ACME', 'Podemos', 'linux', 'BARCOS', 'Hemeroteca', 'Palabros', 'Series', 'Sucesos', 'SysDevs', 'Japón', 'Crianza', 'BUAMBUSUB', 'escombrillos', 'Abuso_Animal', 'Videojuegos', 'Artículos', 'idiomas', 'Música', 'softlibre', 'Visualdata', 'gilipolleces', 'Cocíname', 

In [21]:
#Cogemos los indices de las clases disponibles:
indices_disponibles = frecuencias3.index.values #Filas clases disponibles
# print(indices_disponibles)

indices_no_disponibles = []#Filas clases no disponibles 
for i in frecuencias2.index.values:
    if i not in indices_disponibles:
        indices_no_disponibles.append(i)
    
# print(indices_no_disponibles) 

In [22]:
#Resumen datos no clasificacion:
frecuencias4 = frecuencias2.loc[indices_no_disponibles, ]
frecuencias4

,Numero_Noticias,Clases,Numero_Clases,Peso_Relativo,Peso_Total_Clase%
29,135102,[mnm],1,135102,76.172595
28,23933,[actualidad],1,23933,13.493795
27,9544,[cultura],1,9544,5.381055
26,4415,[tecnología],1,4415,2.489245
8,11,[trenes],1,11,0.006202
6,9,[dmnm],1,9,0.005074


In [23]:
#Seleccionamos
condicion = (df2['sub']!= 'mnm') & (df2['sub']!= 'actualidad') & (df2['sub']!= 'cultura') & (df2['sub']!= 'tecnología') & (df2['sub']!= 'trenes') & (df2['sub']!= 'dmnm')




# condicion = (df2['sub']!= 'mnm')
df3 = df2[condicion]
df3

,noticia,sub,extracto
2,"“Lárgame un cilindrín, fotero”",ocio,"‘La gran superproducción‘ es, sin ningún géner..."
29,El capitalismo japonés: librándonos de mirada...,Japón,La Exposición Internacional de Osaka de 1970 s...
30,Republicanos monárquicos,escombrillos,En una reciente opinión publicada en el sub de
42,Buckley: (bendito) apellido maldito,ocio,Una de las lecciones más valiosas que puede ap...
56,Estoy cansado de los magufos manipuladores,BUAMBUSUB,Hay días en que las redes sociales le quitan ...
...,...,...,...
69737,YouPorn: Las estadísticas del porno gratis,emnm,La presencia del contenido XXX en la Web está ...
70237,Multan con 50.000 euros a un banco por inclui...,emnm,La Agencia Española de Protección de Datos aca...
70385,Expertos escolares abogan por separar a los n...,emnm,¿Qué pensaría si su hijo pequeño regresa del c...
70834,"Guía para usar Calibre, administra tu bibliot...",emnm,«Lo primero que tenemos que saber es que Calib...


In [24]:
#Comprobamos cuantas fila con na hemos eliminado
print(df2.shape)
print(df3.shape)

(177363, 3)
(4349, 3)


In [25]:
#Comprobamos el objeto generador
noticia = next(parse('./meneame.csv/data.csv'))
noticia

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (9,10) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


(' Ruptura inesperada de un iceberg gigante desprendido del mayor glaciar antártico vista por satélite (ING) ',
 'cultura',
 'Una animación del iceberg gigante que nació del Glaciar Pine Island en la Antártida Occidental hace poco más de dos meses muestra una ruptura inesperada vista por satélite. -Lo que estamos presenciando en Pine Island es preocupante. Ahora estamos viendo cambios en el comportamiento de la plataforma de hielo, cuando durante 68 años vimos un patrón de avance y retroceso que resultó en el nacimiento de un solo iceberg grande- dice Robert Larter, geofísico marino de British Antarctic Survey. En español: ')

In [26]:
#Analisis exploratorio a partir de df. Seleccionamos las columnas: noticia, meneos, clicks,
#comentarios, votos_positivos, votos_negativos, votos_anonimos.

exploratorio = df.loc[:,['index' ,'noticia', 'meneos', 'clicks', 'comentarios',
       'votos_positivos', 'votos_anonimos', 'votos_negativos']]
exploratorio

,index,noticia,meneos,clicks,comentarios,votos_positivos,votos_anonimos,votos_negativos
0,2869055,Ruptura inesperada de un iceberg gigante desp...,73,1057,10,55,18,2
1,2869761,"El MIT logra hacer escáneres 3D baratos 1,000...",73,772,17,43,30,1
2,2869841,"“Lárgame un cilindrín, fotero”",89,1608,34,53,36,5
3,2869859,El Tribunal Supremo deja en prisión a Junquer...,171,889,90,98,73,1
4,2869823,La Fiscalía pide imputar a Aguirre y Gallardó...,354,353,39,144,210,0
...,...,...,...,...,...,...,...,...
177504,6,Instalar Linux en iPod Mini | GPL Tarragona,29,,sin,29,0,0
177505,4,Entrevista de El Mundo a Jose Antonio Marina,18,,sin,18,0,0
177506,3,Prova xfce 4.2.3!,34,,sin,34,0,0
177507,2,Los 84 errores de noxtrum,61,,5,61,0,0


## 3. PRE-PROCESADO
Realizaremos el siguiente procesado:
- Separar el texto en *tokens*
- Eliminar los *tokens* de tipo *stop-word*, signos de puntuación, signos especiales o espacios
- Lematizar el texto
- Introducimos un espacio después de determinados signos de puntuación (".", "?", "%") para que el tokenizado sea correcto
- Filtramos los *tokens* con una longitud de 1



In [27]:
nlp = spacy.load("es_core_news_lg") #Mejor modelo optimizado para la CPU

#definimos función de normalizado
def normaliza(texto):
    texto = re.sub(r"(\.)|(\?)|(\%)", r"\1 |\2 |\3 ", texto) #añadimos un espacio después de "." y "?"
    doc = nlp(texto)
    tokens = [t for t in doc if
                        len(t.text)>1 and
                       not t.is_stop and
                       not t.is_space and
                       not t.is_punct]#filtramos los tokens que nos interesan
    palabras = []
    for t in tokens:
        palabras.append(t.lemma_) #añadimos lema
    salida = ' '.join(palabras)#junta todos los tokens en un string
    
    return salida

#funcion para quitar acentos y caracteres no ASCII:

def remove_accents(input_str):     
    nfkd_form = unicodedata.normalize('NFKD', input_str)     
    only_ascii = nfkd_form.encode('ASCII', 'ignore')     
    return only_ascii.decode("utf-8", 'ignore')


In [28]:
#Comprobamos que funciona con el elemento 0 de noticia ("noticia")
print(noticia[0])
normaliza(noticia[0])

 Ruptura inesperada de un iceberg gigante desprendido del mayor glaciar antártico vista por satélite (ING) 


'Ruptura inesperado iceberg gigante desprender glaciar antártico vestir satélite ING'

In [29]:
#normalizamos todo el texto de ambas columnas
#noticia1 = [normaliza(i[0]) for i in parse('./meneame.csv/data.csv')]

In [30]:
#Comprobamos que funciona con el elemento 2 de noticia ("extracto")
print(noticia[2])
normaliza(noticia[2])

Una animación del iceberg gigante que nació del Glaciar Pine Island en la Antártida Occidental hace poco más de dos meses muestra una ruptura inesperada vista por satélite. -Lo que estamos presenciando en Pine Island es preocupante. Ahora estamos viendo cambios en el comportamiento de la plataforma de hielo, cuando durante 68 años vimos un patrón de avance y retroceso que resultó en el nacimiento de un solo iceberg grande- dice Robert Larter, geofísico marino de British Antarctic Survey. En español: 


'animación iceberg gigante nacer Glaciar Pine Island Antártida Occidental mesar mostrar ruptura inesperado vestir satélite -Lo presenciar Pine Island preocupante ver cambio comportamiento plataforma helar 68 año ver patrón avanzar retroceso resultar nacimiento iceberg grande- Robert Larter geofísico marinar British Antarctic Survey español'

In [31]:
#normalizamos todo el texto de ambas columnas
#extracto1 = [normaliza(i[2]) for i in parse('./meneame.csv/data.csv')]

In [32]:
#Creacion de corpus y labels 
corpus = df3['extracto'].tolist() + df3['noticia'].tolist()
labels_posibles = pd.unique(df2['sub']).tolist()
labels = df3['sub'].tolist() + df3['sub'].tolist() 

In [33]:
#Vemos cuantas observaciones en total tenemos.
print(f'\nTamaño  TOTAL: {len(corpus)}')
print(f'Tamaño de clase TOTAL: {len(labels)}')



Tamaño  TOTAL: 8698
Tamaño de clase TOTAL: 8698


In [34]:
#Seleccionamos 5000 muestras del cojunto de datos total
corpus_sm, labels_sm = resample(corpus, labels, n_samples=5000, replace=False) 

In [35]:
#Separacion del conjunto de datos P5
train_corpus, test_corpus, train_labels, test_labels = train_test_split(corpus_sm, labels_sm, test_size = 0.30, shuffle = False)

In [36]:
# COMPLETAR
print(f'\nTamaño de TRAIN: {len(train_corpus)}')
print(f'Tamaño de clase TRAIN: {len(train_labels)}')

print(f'\nTamaño de TEST: {len(test_corpus)}')
print(f'Tamaño de clase TEST: {len(test_labels)}')


Tamaño de TRAIN: 3500
Tamaño de clase TRAIN: 3500

Tamaño de TEST: 1500
Tamaño de clase TEST: 1500


In [37]:
#Normalizamos los extractos y los guardamos como una lista en lugar de generator porque 
#lo tenemos que múltiples veces y no queremos tener que normalizar todo el corpus cada vez.
norm_train_corpus = list(map(normaliza, train_corpus))

In [38]:
norm_test_corpus = list(map(normaliza, test_corpus))

## 4. EXTRACCION DE CARACTERISTICAS
Instanciamos los vectorizadores para obtener las características BoW y TF-IDF.  
Usamos el parámetro max_df=0.9 para eliminar los stop-words como las palabras que aparecen al menos en el 90% de los documentos y el parámetro min_df=0.01 para eliminar las palabras que no aparecen al menos en un 1% de los documentos.\

Usamos el modelo `TfidfTransformer` para calcular la matriz TF-IDF a partir del BoW y no tener que repetir todo el entrenamiento.

Para calcular los modelos basados en WV usamos el modelo gloVe pre-entrenado en `spaCy`. Calculamos dos modelos basados en word-vectors:  
* El vector promedio de los WV de todos los tokens con el mismo peso para todas las palabras.  
* Ponderando el WV de cada palabra por el término de frecuencia inversa de documento (IDF).  

Definimos las funciones para calcular estas dos matrices de características:

In [39]:
#BoW
bow_vectorizer = CountVectorizer(min_df=0.01, max_df=0.9)

#Tf-idf
tfidf_vectorizer = TfidfTransformer()

#Funciones de WV.
def averaged_word_vectorizer(corpus):
    '''Aplica la función de cálculo del WE promedio a todos los
    documentos del corpus (cada doc es una lista de tokens)'''
    features = [nlp(doc).vector
                    for doc in corpus]
    return np.array(features)

def tfidf_wtd_avg_word_vectors(doc, word_tfidf_map):
    '''Aplica la función de cálculo del WE ponderado por TF-IDF
    a un documento (como lista de tokens)'''
    tokens = doc.split()

    feature_vector = np.zeros((nlp.vocab.vectors_length,),dtype="float64")
    wts = 0.      
    for word in tokens:
        if nlp.vocab[word].has_vector and word_tfidf_map.get(word, 0): #sólo considera palabras conocidas
            weighted_word_vector = word_tfidf_map[word] * nlp.vocab[word].vector
            wts = wts + 1
            feature_vector = np.add(feature_vector, weighted_word_vector)
    if wts:
        feature_vector = np.divide(feature_vector, wts)
        
    return feature_vector
    
def tfidf_weighted_averaged_word_vectorizer(corpus, word_tfidf_map):
    '''Aplica la función de cálculo del WE ponderado por TF-IDF a todos los
    documentos del corpus (cada doc es una lista de tokens)'''                                       
    features = [tfidf_wtd_avg_word_vectors(doc, word_tfidf_map)
                    for doc in corpus]
    return np.array(features)

In [40]:
# características bag of words
bow_train_features = bow_vectorizer.fit_transform(norm_train_corpus)  
bow_test_features = bow_vectorizer.transform(norm_test_corpus) 

# características tfidf (a partir del BoW)
tfidf_train_features = tfidf_vectorizer.fit_transform(bow_train_features)
tfidf_test_features = tfidf_vectorizer.transform(bow_test_features)    

# características averaged word vector
avg_wv_train_features = averaged_word_vectorizer(norm_train_corpus)                
avg_wv_test_features = averaged_word_vectorizer(norm_test_corpus)      

# características tfidf weighted averaged word vector
word_tfidf_map = {key:value for (key, value) in zip(bow_vectorizer.get_feature_names(), tfidf_vectorizer.idf_)}

tfidf_wv_train_features = tfidf_weighted_averaged_word_vectorizer(norm_train_corpus, word_tfidf_map)

tfidf_wv_test_features = tfidf_weighted_averaged_word_vectorizer(norm_test_corpus, word_tfidf_map)

In [41]:
bow_train_features.shape

(3500, 137)

In [42]:
tfidf_train_features.shape

(3500, 137)

In [43]:
avg_wv_train_features.shape

(3500, 300)

In [44]:
tfidf_wv_train_features.shape

(3500, 300)

## 5. CLASIFICADOR
Aplicamos distintos clasificadores a cada modelo para ver cuál funciona mejor con nuestros datos. Primero definimos unas funciones para entrenar y medir el rendimiento de los clasificadores:

In [45]:
def get_metrics(true_labels, predicted_labels):
    """Calculamos distintas métricas sobre el
    rendimiento del modelo. Devuelve un diccionario
    con los parámetros medidos"""
    
    return {
        'Accuracy': np.round(
                        metrics.accuracy_score(true_labels, 
                                               predicted_labels),
                        3),
        'Precision': np.round(
                        metrics.precision_score(true_labels, 
                                               predicted_labels,
                                               average='weighted',
                                               zero_division=0),
                        3),
    'Recall': np.round(
                        metrics.recall_score(true_labels, 
                                               predicted_labels,
                                               average='weighted',
                                               zero_division=0),
                        3),
    'F1 Score': np.round(
                        metrics.f1_score(true_labels, 
                                               predicted_labels,
                                               average='weighted',
                                               zero_division=0),
                        3)}
                        

def train_predict_evaluate_model(classifier, 
                                 train_features, train_labels, 
                                 test_features, test_labels):
    """Función que entrena un modelo de clasificación sobre
    un conjunto de entrenamiento, lo aplica sobre un conjunto
    de test y devuelve la predicción sobre el conjunto de test
    y las métricas de rendimiento"""
    # genera modelo    
    classifier.fit(train_features, train_labels)
    # predice usando el modelo sobre test
    predictions = classifier.predict(test_features) 
    # evalúa rendimiento de la predicción   
    metricas = get_metrics(true_labels=test_labels, 
                predicted_labels=predictions)
    return predictions, metricas     

Vamos a entrenar sobre el conjunto de train y evaluamos en el conjunto de test. Guardamos métricas en una lista y resultados en otra para mostrar resumen.

In [46]:
modelLR = LogisticRegression(solver='liblinear')
modelNB = GaussianNB()
modelSVM = SGDClassifier(loss='hinge', max_iter=1000)
modelRBFSVM = SVC(gamma='scale', C=2)

modelos = [('Logistic Regression', modelLR),
           ('Naive Bayes', modelNB),
           ('Linear SVM', modelSVM),
           ('Gauss kernel SVM', modelRBFSVM)]

metricas = []
resultados = []

# Modelos con características BoW
bow_train_features_ = bow_train_features.toarray()
bow_test_features_ = bow_test_features.toarray()
for m, clf in modelos:
    prediccion, metrica = train_predict_evaluate_model(classifier=clf,
                                           train_features=bow_train_features_,
                                           train_labels=train_labels,
                                           test_features=bow_test_features_,
                                           test_labels=test_labels)
    metrica['modelo']=f'{m} BoW'
    resultados.append(prediccion)
    metricas.append(metrica)
    
# Modelos con características TF-IDF
tfidf_train_features_ = tfidf_train_features.toarray()
tfidf_test_features_ = tfidf_test_features.toarray()
for m, clf in modelos:
    prediccion, metrica = train_predict_evaluate_model(classifier=clf,
                                           train_features=tfidf_train_features_,
                                           train_labels=train_labels,
                                           test_features=tfidf_test_features_,
                                           test_labels=test_labels)
    metrica['modelo']=f'{m} tfidf'
    resultados.append(prediccion)
    metricas.append(metrica)

# Modelos con características averaged word vectors
avg_wv_train_features_ = avg_wv_train_features
avg_wv_test_features_ = avg_wv_test_features
for m, clf in modelos:
    prediccion, metrica = train_predict_evaluate_model(classifier=clf,
                                           train_features=avg_wv_train_features_,
                                           train_labels=train_labels,
                                           test_features=avg_wv_test_features_,
                                           test_labels=test_labels)
    metrica['modelo']=f'{m} averaged'
    resultados.append(prediccion)
    metricas.append(metrica)

# Modelos con características tfidf weighted averaged word vectors
tfidf_wv_train_features_ = tfidf_wv_train_features
tfidf_wv_test_features_ = tfidf_wv_test_features
for m, clf in modelos:
    prediccion, metrica = train_predict_evaluate_model(classifier=clf,
                                           train_features=tfidf_wv_train_features_,
                                           train_labels=train_labels,
                                           test_features=tfidf_wv_test_features_,
                                           test_labels=test_labels)
    metrica['modelo']=f'{m} tfidf wv'
    resultados.append(prediccion)
    metricas.append(metrica)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Conviertimos la lista de métricas en un DataFrame para observar sus valores:

In [47]:
metricas = pd.DataFrame(metricas)
metricas

,Accuracy,Precision,Recall,F1 Score,modelo
0,0.805,0.663,0.805,0.724,Logistic Regression BoW
1,0.233,0.686,0.233,0.343,Naive Bayes BoW
2,0.807,0.665,0.807,0.727,Linear SVM BoW
3,0.808,0.668,0.808,0.725,Gauss kernel SVM BoW
4,0.807,0.663,0.807,0.723,Logistic Regression tfidf
5,0.234,0.687,0.234,0.344,Naive Bayes tfidf
6,0.808,0.663,0.808,0.725,Linear SVM tfidf
7,0.805,0.687,0.805,0.726,Gauss kernel SVM tfidf
8,0.797,0.733,0.797,0.759,Logistic Regression averaged
9,0.263,0.731,0.263,0.346,Naive Bayes averaged


Ordenamos las métricas por `accuracy` y muestra el mejor resultado:

In [48]:
metricas = metricas.sort_values('Accuracy',ascending=False)
metricas.head(1)

,Accuracy,Precision,Recall,F1 Score,modelo
11,0.83,0.729,0.83,0.765,Gauss kernel SVM averaged


Mejoramos el `accuracy` a partir del juego de parametros

In [49]:
#https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html
#Funcion de gamma
gamma = ['scale', 'auto']

metricas = []
resultados = []


for i in gamma:
    modelRBFSVM = SVC(gamma=i, C=2)
    prediccion, metrica = train_predict_evaluate_model(classifier=modelRBFSVM,
                                           train_features=avg_wv_train_features_,
                                           train_labels=train_labels,
                                           test_features=avg_wv_test_features_,
                                           test_labels=test_labels)
    metrica['modelo']=f'{i} – Gauss kernel SVM averaged'
    resultados.append(prediccion)
    metricas.append(metrica)

Conviertimos la lista de métricas en un DataFrame para observar sus valores:

In [50]:
metricas = pd.DataFrame(metricas)
metricas

,Accuracy,Precision,Recall,F1 Score,modelo
0,0.830,0.729,0.830,0.765,scale – Gauss kernel SVM averaged
1,0.825,0.724,0.825,0.758,auto – Gauss kernel SVM averaged


Ordenamos las métricas por `accuracy` y muestra el mejor resultado:

In [51]:
metricas = metricas.sort_values('Accuracy',ascending=False)
metricas.head(1)

,Accuracy,Precision,Recall,F1 Score,modelo
0,0.83,0.729,0.83,0.765,scale – Gauss kernel SVM averaged


Dividimos el conjunto total de muestras (`corpus` y `labels`) en entrenamiento y test (30%) y entrenamos el mejor modelo obtenido, para ver si se mejoran los resultados:

In [52]:
## MEJOR MODELO
train_corpus, test_corpus, train_labels, test_labels = train_test_split(corpus, labels, test_size = 0.30)


In [53]:
#Normalizamos
norm_train_corpus = list(map(normaliza, train_corpus))
norm_test_corpus = list(map(normaliza, test_corpus))



In [54]:
# Características averaged word vector
avg_wv_train_features = averaged_word_vectorizer(norm_train_corpus)                
avg_wv_test_features = averaged_word_vectorizer(norm_test_corpus)      
avg_wv_train_features_ = avg_wv_train_features
avg_wv_test_features_ = avg_wv_test_features

In [55]:
# Modelos con características
modelRBFSVM = SVC(gamma='scale', C=2)
prediccion, metrica = train_predict_evaluate_model(classifier=modelRBFSVM,
                                           train_features=avg_wv_train_features_,
                                           train_labels=train_labels,
                                           test_features=avg_wv_test_features_,
                                           test_labels=test_labels)



In [56]:
print('Modelo Gauss kernel SVM averaged con gamma scale: ')
for i in metrica:
    print('- ',i, ': ', metrica[i])

print('\nPredicciones:')
print(prediccion)

Modelo Gauss kernel SVM averaged con gamma scale: 
-  Accuracy :  0.828
-  Precision :  0.727
-  Recall :  0.828
-  F1 Score :  0.763

Predicciones:
['ocio' 'ocio' 'ocio' ... 'ocio' 'ocio' 'ocio']


In [57]:
pd.unique(prediccion)

array(['ocio', 'astronomia', 'ciencia', 'TeRespondo', 'linux',
       'Pregúntame', 'emnm', 'Palabros', 'Jurídicas', 'BARCOS'],
      dtype='<U12')